In [1]:
import pandas as pd
import numpy as np
import pickle
import polars as pl

In [2]:
def load_targets(target):
    targets = pd.read_csv(target)
    targets["session"] = targets["session_id"].str.split("_",expand = True)[0]
    targets["session"] = targets["session"].astype(int)
    targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
    return targets

In [2]:
target_path = '../data/raw/input/train_labels.csv'
data_path = '../data/raw/collected/more_data.parquet'
# targets = load_targets(target_path)
df = pd.read_parquet(data_path)

In [3]:
pickle.dump(df, open('../data/processed/concat/train.pkl', 'wb'))

In [4]:
# add_df = pd.read_parquet('../data/raw/collected/more_data.parquet')
add_target = pd.read_parquet('../data/raw/collected/more_label.parquet')

# display(add_df.head())
# display(add_target.head())
add_target

,session_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,save_code
0,19110000090781600,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0
1,19110003194626970,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,19110007435189656,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,19110008053486390,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,19110008122038204,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,22060612030771748,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
605,22060614142472616,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
606,22060615522168090,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0
607,22060616521475370,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [7]:
# 1. target 18개 있는 세션만 저장하기
df.set_index('session_id').loc[add_target.dropna().session_id].reset_index().to_parquet('../data/processed/concat/more_train_fully_labeled.parquet')

## 1. NaN 값을 다 빼서 10000명으로만 유저 데이터를 합쳐서 학습 진행 - 시도 - 성능 안오른다 - 버리는걸로.. 


In [2]:
# more_train = pd.read_parquet('../data/processed/concat/more_train_fully_labeled.parquet')
# train = pd.read_parquet('../data/raw/input/train.parquet')

total_train = pd.concat([pd.read_parquet('../data/raw/input/train.parquet'),
                          pd.read_parquet('../data/processed/concat/more_train_fully_labeled.parquet').drop(columns=['save_code'])])\
                .astype({
                        'elapsed_time': 'int64',
                        'level': 'int8'
                    })\
                .to_parquet('../data/processed/concat/total_train_fully_labeled.parquet')

In [5]:
# target
target = pd.read_csv('../data/raw/input/train_labels.csv')
target

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1
...,...,...
424111,22100215342220508_q18,1
424112,22100215460321130_q18,1
424113,22100217104993650_q18,1
424114,22100219442786200_q18,1


In [33]:
man_target

,session_id,q,correct
0,19110000090781600,0,1.0
1,19110009114094480,0,1.0
2,19110009240104090,0,1.0
3,19110011340239490,0,1.0
4,19110012454548376,0,0.0
...,...,...,...
232717,22060512165143500,17,1.0
232718,22060516353729760,17,0.0
232719,22060518310026576,17,1.0
232720,22060610361404176,17,1.0


In [40]:
man_target = add_target.dropna().drop(columns=['save_code']).melt('session_id', var_name='q', value_name='correct')
man_target['session_id'] = man_target.apply(lambda x: str(x['session_id']) + '_q' + str(int(x['q'])+1), axis=1)
del man_target['q']
man_target

,session_id,correct
0,19110000090781600_q1,1.0
1,19110009114094480_q1,1.0
2,19110009240104090_q1,1.0
3,19110011340239490_q1,1.0
4,19110012454548376_q1,0.0
...,...,...
232717,22060512165143500_q18,1.0
232718,22060516353729760_q18,0.0
232719,22060518310026576_q18,1.0
232720,22060610361404176_q18,1.0


In [45]:
target

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1
...,...,...
424111,22100215342220508_q18,1
424112,22100215460321130_q18,1
424113,22100217104993650_q18,1
424114,22100219442786200_q18,1


In [55]:
man_target

,session_id,correct
0,19110000090781600_q1,1
1,19110009114094480_q1,1
2,19110009240104090_q1,1
3,19110011340239490_q1,1
4,19110012454548376_q1,0
...,...,...
232717,22060512165143500_q18,1
232718,22060516353729760_q18,0
232719,22060518310026576_q18,1
232720,22060610361404176_q18,1


In [51]:
man_target = man_target.astype({'correct': 'int64'})

In [61]:
print(target.dtypes)
print(man_target.dtypes)

session_id    object
correct        int64
dtype: object
session_id    object
correct        int64
dtype: object


In [ ]:
target.merge()

In [84]:
mtarget = target.merge(man_target, on='session_id', how='outer')
mtarget['correct'] = mtarget.apply(lambda x: x['correct_x'] if ~np.isnan(x['correct_x']) else x['correct_y'], axis=1)
del mtarget['correct_x'], mtarget['correct_y']
mtarget.to_parquet('../data/processed/concat/total_target_full_labeled.parquet')

In [82]:
mtarget.

numpy.float64

In [70]:
mtarget[(mtarget['correct_x'] != mtarget['correct_y'])]

,session_id,correct_x,correct_y
16,21000521512037530_q1,1,0
25,22000009242414744_q1,1,0
27,22000009400806260_q1,1,0
32,22000010572263176_q1,1,0
75,22000019560697010_q1,1,0
...,...,...,...
93069,22090512564049490_q18,1,0
93089,22090516154433520_q18,1,0
93130,22090613301481920_q18,1,0
93157,22090618154452916_q18,1,0


In [20]:
add_target[add_target.columns[1:-1]].to_numpy()
mani_target = add_target[['session_id']]
mani_target

,session_id
0,19110000090781600
1,19110003194626970
2,19110007435189656
3,19110008053486390
4,19110008122038204
...,...
604,22060612030771748
605,22060614142472616
606,22060615522168090
607,22060616521475370


## 2. 부분적으로 정답 데이터가 있으면 완전히 라벨이 있는 레벨그룹의 데이터는 살려두기 - 

In [8]:
pd.read_parquet('../data/processed/concat/more_train_fully_labeled.parquet')

,session_id,index,fqid,level,name,room_fqid,fullscreen,hq,music,text,...,page,save_code,hover_duration,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,event_name,level_group,elapsed_time
0,19110000090781600,0,intro,0.0,basic,tunic.historicalsociety.closet,NaN,NaN,NaN,undefined,...,NaN,NaN,NaN,-421.991405,137.685314,372.0,197.0,cutscene_click,0-4,0.000000e+00
1,19110000090781600,1,gramps,0.0,basic,tunic.historicalsociety.closet,NaN,NaN,NaN,"Whatcha doing over there, Jo?",...,NaN,NaN,NaN,-421.991405,137.685314,372.0,197.0,person_click,0-4,4.670000e+02
2,19110000090781600,2,gramps,0.0,basic,tunic.historicalsociety.closet,NaN,NaN,NaN,Just talking to Teddy.,...,NaN,NaN,NaN,-294.991405,-9.314686,499.0,344.0,person_click,0-4,1.732000e+03
3,19110000090781600,3,gramps,0.0,basic,tunic.historicalsociety.closet,NaN,NaN,NaN,I gotta run to my meeting!,...,NaN,NaN,NaN,-294.991405,-9.314686,499.0,344.0,person_click,0-4,2.965000e+03
4,19110000090781600,4,gramps,0.0,basic,tunic.historicalsociety.closet,NaN,NaN,NaN,Meetings are BORING!,...,NaN,NaN,NaN,-294.991405,-9.314686,499.0,344.0,person_click,0-4,4.282000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14690378,22060615522168090,1650,tomap,22.0,undefined,tunic.historicalsociety.entry,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,navigate_click,13-22,1.943741e+06
14690379,22060615522168090,1653,tunic.capitol_2,22.0,undefined,tunic.historicalsociety.entry,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,map_click,13-22,1.945772e+06
14690380,22060615522168090,1654,chap4_finale_c,22.0,undefined,tunic.capitol_2.hall,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,navigate_click,13-22,1.946870e+06
14690381,22060615522168090,1656,chap4_finale_c,22.0,undefined,tunic.capitol_2.hall,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,navigate_click,13-22,1.948052e+06


연관성 있는 거는 놔두고,  
1. feature importance 측정해서 낮은거를 빼서 시도 
2. 

In [5]:
pickle.dump(add_df, open('../data/processed/concat/more_train.pkl', 'wb'))

In [5]:
import gc
gc.collect()

18

In [2]:
more_path = '../data/processed/concat/more_train.pkl'
train_path = '../data/processed/concat/train.pkl'
# dfs = pd.concat([pd.read_parquet(data_path), pd.read_parquet('../data/raw/collected/more_data.parquet').drop(columns=['save_code'])])
dfs = pd.concat(
    [
        pickle.load(open(train_path, 'rb')), pickle.load(open(more_path, 'rb')).drop(columns=['save_code'])
    ]
)
# df_list.append()
dfs

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0.000000,cutscene_click,basic,0.0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0.0,0.0,1.0,0-4
1,20090312431273200,1,1323.000000,person_click,basic,0.0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0.0,0.0,1.0,0-4
2,20090312431273200,2,831.000000,person_click,basic,0.0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0.0,0.0,1.0,0-4
3,20090312431273200,3,1147.000000,person_click,basic,0.0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0.0,0.0,1.0,0-4
4,20090312431273200,4,1863.000000,person_click,basic,0.0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0.0,0.0,1.0,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537433,22090621002910536,602,533742.000000,navigate_click,undefined,12.0,NaN,NaN,NaN,NaN,NaN,NaN,None,toentry,tunic.historicalsociety.stacks,None,NaN,NaN,NaN,5-12
1537434,22090621002910536,606,535989.000000,navigate_click,undefined,12.0,NaN,NaN,NaN,NaN,NaN,NaN,None,tomap,tunic.historicalsociety.entry,None,NaN,NaN,NaN,5-12
1537435,22090621002910536,610,538972.000000,map_click,undefined,12.0,NaN,NaN,NaN,NaN,NaN,NaN,None,tunic.capitol_1,tunic.historicalsociety.entry,None,NaN,NaN,NaN,5-12
1537436,22090621002910536,611,542015.000244,navigate_click,undefined,12.0,NaN,NaN,NaN,NaN,NaN,NaN,None,chap2_finale_c,tunic.capitol_1.hall,None,NaN,NaN,NaN,5-12


In [2]:
dfs = pl.from_pandas(pickle.load(open('../data/processed/concat/total_train.pkl', 'rb')))

In [2]:
# dfs = pd.read_parquet('../data/processed/concat/total_train.parquet')

: 

: 

In [4]:
dfs

session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
i64,i64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,str
20090312431273200,0,0.0,"""cutscene_click…","""basic""",0.0,null,-413.991405,-159.314686,380.0,494.0,null,"""undefined""","""intro""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,1,1323.0,"""person_click""","""basic""",0.0,null,-413.991405,-159.314686,380.0,494.0,null,"""Whatcha doing …","""gramps""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,2,831.0,"""person_click""","""basic""",0.0,null,-413.991405,-159.314686,380.0,494.0,null,"""Just talking t…","""gramps""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,3,1147.0,"""person_click""","""basic""",0.0,null,-413.991405,-159.314686,380.0,494.0,null,"""I gotta run to…","""gramps""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,4,1863.0,"""person_click""","""basic""",0.0,null,-412.991405,-159.314686,381.0,494.0,null,"""Can I come, Gr…","""gramps""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,5,3423.0,"""person_click""","""basic""",0.0,null,-412.991405,-157.314686,381.0,492.0,null,"""Sure thing, Jo…","""gramps""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,6,5197.0,"""person_click""","""basic""",0.0,null,478.485079,-199.971679,593.0,485.0,null,"""See you later,…","""teddy""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,7,6180.0,"""person_click""","""basic""",0.0,null,503.355128,-168.619913,609.0,453.0,null,"""I get to go to…","""teddy""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""
20090312431273200,8,7014.0,"""person_click""","""basic""",0.0,null,510.733442,-157.720642,615.0,442.0,null,"""Now where did …","""teddy""","""tunic.historic…","""tunic.historic…",0.0,0.0,1.0,"""0-4"""


In [3]:
target = pl.from_pandas(pd.read_parquet('../data/raw/collected/more_label.parquet'))
target

session_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,save_code
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
19110000090781600,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,"""0"""
19110003194626970,0.0,1.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0"""
19110007435189656,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,"""0.0"""
19110008053486390,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,"""0.0"""
19110008122038204,1.0,1.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0"""
19110008192774350,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,null,null,null,null,null,"""notebook"""
19110008244001480,0.0,1.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0"""
19110008462915572,1.0,1.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""0.0"""
19110008501053136,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,null,null,null,null,null,"""0.0"""


In [41]:
dfs_part = dfs[-10000:]
dfs_part

session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
i64,i64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,str
22090610131603720,1415,1.579023e6,"""cutscene_click…","""basic""",16.0,null,null,null,null,null,null,"""Alright, Jolie…","""confrontation""","""tunic.historic…","""tunic.historic…",null,null,null,"""13-22"""
22090610131603720,1416,1.580623e6,"""cutscene_click…","""basic""",16.0,null,null,null,null,null,null,"""Your grampa is…","""confrontation""","""tunic.historic…","""tunic.historic…",null,null,null,"""13-22"""
22090610131603720,1417,1.581854e6,"""cutscene_click…","""basic""",16.0,null,null,null,null,null,null,"""Come on, Teddy…","""confrontation""","""tunic.historic…","""tunic.historic…",null,null,null,"""13-22"""
22090610131603720,1418,1.5827e6,"""cutscene_click…","""basic""",16.0,null,null,null,null,null,null,"""Let's go find …","""confrontation""","""tunic.historic…","""tunic.historic…",null,null,null,"""13-22"""
22090610131603720,1420,1.583724e6,"""navigate_click…","""undefined""",17.0,null,null,null,null,null,null,null,"""0""","""tunic.historic…",null,null,null,null,"""13-22"""
22090610131603720,1421,1.584092e6,"""navigate_click…","""undefined""",17.0,null,null,null,null,null,null,null,"""0""","""tunic.historic…",null,null,null,null,"""13-22"""
22090610131603720,1422,1.584307e6,"""navigate_click…","""undefined""",17.0,null,null,null,null,null,null,null,"""0""","""tunic.historic…",null,null,null,null,"""13-22"""
22090610131603720,1423,1.584511e6,"""navigate_click…","""undefined""",17.0,null,null,null,null,null,null,null,"""0""","""tunic.historic…",null,null,null,null,"""13-22"""
22090610131603720,1424,1.584691e6,"""navigate_click…","""undefined""",17.0,null,null,null,null,null,null,null,"""0""","""tunic.historic…",null,null,null,null,"""13-22"""


In [39]:
# len(set(dfs['session_id'].to_numpy()))# & set(target['session_id'].to_numpy())

# len(set(target['session_id'].to_numpy()))

57121

In [20]:
target.drop_nulls().select('session_id')

session_id
i64
19110000090781600
19110009114094480
19110009240104090
19110011340239490
19110012454548376
19110013371836976
19110015213957736
19110015442803260
19110016345644932


In [43]:
pickle.dump(dfs.join(target.drop_nulls().select('session_id'), on='session_id', how='inner').to_pandas(), open('../data/processed/concat/more_train_fully_labeled.pkl', 'wb'))

: 

: 

In [4]:
# target 값 모두 존재하는 경우만 남겨두기 
pickle.dump(dfs.set_index('session_id').loc[add_target.dropna().session_id], open('../data/processed/concat/total_train_finusers', 'wb'))

: 

: 

In [5]:
dfs['text_fqid'].unique()

array(['tunic.historicalsociety.closet.intro',
       'tunic.historicalsociety.closet.gramps.intro_0_cs_0',
       'tunic.historicalsociety.closet.teddy.intro_0_cs_0', None,
       'tunic.historicalsociety.closet.teddy.intro_0_cs_5',
       'tunic.historicalsociety.closet.photo',
       'tunic.historicalsociety.closet.notebook',
       'tunic.historicalsociety.closet.retirement_letter.hub',
       'tunic.historicalsociety.basement.janitor',
       'tunic.historicalsociety.entry.groupconvo',
       'tunic.historicalsociety.entry.boss.talktogramps',
       'tunic.historicalsociety.entry.wells.talktogramps',
       'tunic.historicalsociety.collection.cs',
       'tunic.historicalsociety.collection.tunic.slip',
       'tunic.historicalsociety.collection.gramps.found',
       'tunic.historicalsociety.stacks.outtolunch',
       'tunic.kohlcenter.halloffame.plaque.face.date',
       'tunic.kohlcenter.halloffame.togrampa',
       'tunic.capitol_0.hall.boss.talktogramps',
       'tunic.historic

In [6]:
dfs['fqid'].unique()

array(['intro', 'gramps', 'teddy', 'photo', None, 'notebook',
       'retirement_letter', 'tobasement', 'janitor', 'toentry',
       'groupconvo', 'report', 'boss', 'wells', 'directory',
       'tocollection', 'cs', 'tunic', 'tunic.hub.slip', 'tostacks',
       'outtolunch', 'tocloset', 'tomap', 'tunic.historicalsociety',
       'tunic.kohlcenter', 'plaque', 'plaque.face.date', 'togrampa',
       'tunic.capitol_0', 'chap1_finale', 'chap1_finale_c',
       'tocloset_dirty', 'what_happened', 'trigger_scarf',
       'trigger_coffee', 'tunic.capitol_1', 'tofrontdesk', 'archivist',
       'magnify', 'tunic.humanecology', 'worker', 'businesscards',
       'businesscards.card_0.next', 'businesscards.card_1.next',
       'businesscards.card_bingo.next', 'businesscards.card_bingo.bingo',
       'tohallway', 'tunic.drycleaner', 'logbook', 'logbook.page.bingo',
       'tunic.library', 'tomicrofiche', 'reader', 'reader.paper0.next',
       'reader.paper1.next', 'reader.paper2.bingo', 'wellsbadge',

In [ ]:
add_target = pd.read_parquet('../data/raw/collected/more_label.parquet')
add_target

,session_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,save_code
0,19110000090781600,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0
1,19110003194626970,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,19110007435189656,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,19110008053486390,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,19110008122038204,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,22060612030771748,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
605,22060614142472616,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
606,22060615522168090,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0
607,22060616521475370,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [14]:
add_target.sample(10)

,session_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,save_code
259,22060113190074090,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2379,20010617204263936,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3369,22000308205590510,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,22060110285634984,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0
1172,20010312544056830,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2110,22090409512255504,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2484,20040411184960390,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3867,22030210072492556,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2364,20020512202774744,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
632,22100208531800390,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [18]:
dfs.text_fqid[10000]

10000          tunic.humanecology.frontdesk.worker.intro
10000    tunic.historicalsociety.collection.gramps.found
10000                                               None
10000                                               None
10000         tunic.historicalsociety.cage.confrontation
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000                                               None
10000              tunic.library.frontdesk.worker.nelson
10000           tunic.historicalsociety.entry.groupconvo
10000                                               None
10000                          

In [ ]:
dfs['fqid'].unique()

In [20]:
dfs.loc[dfs.text_fqid == 'tunic.historicalsociety.cage.confrontation', 'fqid'].unique()

array(['confrontation'], dtype=object)

In [15]:
## nan값만 없으면 이렇게 압축 가능함..
# dfs.astype({
#     'elapsed_time': 'int64',
#     'level': 'int8',
#     'page': 'int8', 
#     'room_coor_x': 'int32',
#     'room_coor_y': 'int32',
#     'screen_coor_x': 'int32',
#     'screen_coor_y': 'int32',
#     'hover_duration': 'int64',
#     'fullscreen': 'int8', 
#     'hq': 'int8',
#     'music': 'int8'
# }).to_parquet('../data/processed/concat/more_train_comp.parquet')

In [6]:
pd.concat([df, add_df.drop(columns=['save_code'])])


: 

: 

In [ ]:
# 원래 데이터에 있는걸 우선으로
def add_data(df, targets):
    dfa = pd.concat([df, add_df.drop(columns=['save_code'])])
    return 

# total_df, total_targets = add_data(df, targets)